<a href="https://colab.research.google.com/github/shramana263/ResoluteAI-Python/blob/main/Crowd_detection_in_videos(Using%20YOLO%20and%20DBSCAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
from scipy.optimize import linear_sum_assignment
import torch
from yolov5 import YOLOv5

# Initialize YOLOv5 model
model = YOLOv5('yolov5s.pt', device='cuda' if torch.cuda.is_available() else 'cpu')

# Video capture
cap = cv2.VideoCapture('input_video.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
epsilon = 0.05 * frame_width  # DBSCAN distance threshold
min_samples = 3  # Minimum persons for a crowd

# Data structures
groups = {}  # {group_id: {'frames': [], 'sizes': [], 'consecutive_count': 0, 'last_frame': -1}}
next_group_id = 0
logs = []  # List to store (frame_number, person_count)

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect persons
    results = model.predict(frame)
    detections = results.pred[0]
    person_detections = [d for d in detections if int(d[5]) == 0]  # COCO class 0 is 'person'

    # Extract bounding box centers
    centers = []
    for det in person_detections:
        x1, y1, x2, y2 = det[:4].cpu().numpy()
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        centers.append([center_x, center_y])
    centers = np.array(centers)

    # Cluster persons using DBSCAN
    if len(centers) > 0:
        clustering = DBSCAN(eps=epsilon, min_samples=min_samples).fit(centers)
        labels = clustering.labels_
    else:
        labels = []

    # Form groups
    current_groups = {}
    for label in set(labels):
        if label == -1:  # Noise points
            continue
        group_centers = centers[labels == label]
        group_size = len(group_centers)
        if group_size >= min_samples:
            group_center = np.mean(group_centers, axis=0)
            current_groups[label] = {'center': group_center, 'size': group_size}

    # Match current groups with previous groups
    if groups:
        prev_group_ids = list(groups.keys())
        curr_group_labels = list(current_groups.keys())
        cost_matrix = np.zeros((len(prev_group_ids), len(curr_group_labels)))
        for i, gid in enumerate(prev_group_ids):
            prev_center = groups[gid]['center']
            for j, label in enumerate(curr_group_labels):
                curr_center = current_groups[label]['center']
                cost_matrix[i, j] = np.linalg.norm(prev_center - curr_center)

        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matched = {}
        for i, j in zip(row_ind, col_ind):
            if cost_matrix[i, j] < 2 * epsilon:
                matched[curr_group_labels[j]] = prev_group_ids[i]

    # Update groups
    new_groups = {}
    for label in current_groups:
        if label in matched:
            gid = matched[label]
            group = groups[gid]
            if frame_number == group['last_frame'] + 1:
                group['consecutive_count'] += 1
            else:
                group['consecutive_count'] = 1
            group['frames'].append(frame_number)
            group['sizes'].append(current_groups[label]['size'])
            group['center'] = current_groups[label]['center']
            group['last_frame'] = frame_number
            new_groups[gid] = group
            if group['consecutive_count'] >= 10:
                logs.append({'Frame Number': frame_number, "Person Count in Crowd": current_groups[label]['size']})
        else:
            new_groups[next_group_id] = {
                'frames': [frame_number],
                'sizes': [current_groups[label]['size']],
                'center': current_groups[label]['center'],
                'consecutive_count': 1,
                'last_frame': frame_number
            }
            next_group_id += 1
    groups = new_groups

    frame_number += 1

# Save logs to CSV
if logs:
    df = pd.DataFrame(logs)
    df.to_csv('crowd_detection.csv', index=False)

cap.release()

ModuleNotFoundError: No module named 'yolov5'

In [ ]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━

In [9]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
from scipy.optimize import linear_sum_assignment
import torch
from yolov5 import YOLOv5
import re
import warnings
from tqdm import tqdm

# Suppress YOLOv5 deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="yolov5.*")

# Function to get file ID from Google Drive shareable link
def get_file_id_from_link(link):
    match = re.search(r'/d/(.*?)/view', link)
    if match:
        return match.group(1)
    else:
        return None

# Function to get direct download URL
def get_direct_download_url(file_id):
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Get user input for Google Drive link
user_link = input("Please enter the Google Drive shareable link: ")
file_id = get_file_id_from_link(user_link)
if file_id is None:
    print("Invalid Google Drive link. Please provide a valid shareable link.")
    exit()

direct_url = get_direct_download_url(file_id)

# Initialize YOLOv5 model
model = YOLOv5('yolov5s.pt', device='cuda' if torch.cuda.is_available() else 'cpu')

# Video capture from URL
cap = cv2.VideoCapture(direct_url)
if not cap.isOpened():
    print("Error: Could not open video capture. Please ensure the file is accessible and set to 'Anyone with the link'.")
    exit()

# Get total frame count for progress bar
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
epsilon = 0.05 * frame_width  # DBSCAN distance threshold
min_samples = 3  # Minimum persons for a crowd

# Data structures
groups = {}  # {group_id: {'frames': [], 'sizes': [], 'consecutive_count': 0, 'last_frame': -1}}
next_group_id = 0
logs = []  # List to store (frame_number, person_count)

frame_number = 0
with tqdm(total=total_frames, desc="Processing frames") as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print(f"Debug: Video capture ended at frame {frame_number} (ret=False)")
            break

        # Detect persons
        results = model.predict(frame)
        detections = results.pred[0]
        person_detections = [d for d in detections if int(d[5]) == 0]  # COCO class 0 is 'person'
        print(f"Debug: Frame {frame_number} - {len(person_detections)} persons detected")

        # Extract bounding box centers
        centers = []
        for det in person_detections:
            x1, y1, x2, y2 = det[:4].cpu().numpy()
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            centers.append([center_x, center_y])
        centers = np.array(centers)

        # Cluster persons using DBSCAN
        if len(centers) > 0:
            clustering = DBSCAN(eps=epsilon, min_samples=min_samples).fit(centers)
            labels = clustering.labels_
            num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            print(f"Debug: Frame {frame_number} - {num_clusters} clusters found")
        else:
            labels = []
            print(f"Debug: Frame {frame_number} - No clusters (no centers)")

        # Form groups
        current_groups = {}
        for label in set(labels):
            if label == -1:  # Noise points
                continue
            group_centers = centers[labels == label]
            group_size = len(group_centers)
            if group_size >= min_samples:
                group_center = np.mean(group_centers, axis=0)
                current_groups[label] = {'center': group_center, 'size': group_size}

        # Match current groups with previous groups
        matched = {}
        if groups and current_groups:
            prev_group_ids = list(groups.keys())
            curr_group_labels = list(current_groups.keys())
            cost_matrix = np.zeros((len(prev_group_ids), len(curr_group_labels)))
            for i, gid in enumerate(prev_group_ids):
                prev_center = groups[gid]['center']
                for j, label in enumerate(curr_group_labels):
                    curr_center = current_groups[label]['center']
                    cost_matrix[i, j] = np.linalg.norm(prev_center - curr_center)

            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            for i, j in zip(row_ind, col_ind):
                if cost_matrix[i, j] < 2 * epsilon:
                    matched[curr_group_labels[j]] = prev_group_ids[i]

        # Update groups
        new_groups = {}
        for label in current_groups:
            if label in matched:
                gid = matched[label]
                group = groups[gid]
                if frame_number == group['last_frame'] + 1:
                    group['consecutive_count'] += 1
                else:
                    group['consecutive_count'] = 1
                group['frames'].append(frame_number)
                group['sizes'].append(current_groups[label]['size'])
                group['center'] = current_groups[label]['center']
                group['last_frame'] = frame_number
                new_groups[gid] = group
                if group['consecutive_count'] >= 10:
                    logs.append({'Frame Number': frame_number, "Person Count in Crowd": current_groups[label]['size']})
                    print(f"Debug: Frame {frame_number} - Crowd detected (size: {current_groups[label]['size']})")
            else:
                new_groups[next_group_id] = {
                    'frames': [frame_number],
                    'sizes': [current_groups[label]['size']],
                    'center': current_groups[label]['center'],
                    'consecutive_count': 1,
                    'last_frame': frame_number
                }
                next_group_id += 1
        groups = new_groups

        frame_number += 1
        pbar.update(1)

# Save logs to CSV
if logs:
    df = pd.DataFrame(logs)
    df.to_csv('crowd_detection.csv', index=False)
    print("Debug: CSV file 'crowd_detection.csv' generated with crowd detections")
else:
    # Write empty CSV with headers
    df = pd.DataFrame(columns=['Frame Number', 'Person Count in Crowd'])
    df.to_csv('crowd_detection.csv', index=False)
    print("Debug: No crowds detected; empty CSV file 'crowd_detection.csv' generated")

cap.release()
print("Debug: Script completed processing")

#sample link:
#with crowd- https://drive.google.com/file/d/1uY-eQOvFgmwnbmvLVRIXxfh1wepK4OdF/view?usp=drive_link
#without crowd- https://drive.google.com/file/d/161VKGD6AnjzqmozcOhX6mu2RBRhXS9XO/view?usp=sharing

Please enter the Google Drive shareable link: https://drive.google.com/file/d/1uY-eQOvFgmwnbmvLVRIXxfh1wepK4OdF/view?usp=drive_link


Processing frames:   0%|          | 1/341 [00:00<02:37,  2.15it/s]

Debug: Frame 0 - 37 persons detected
Debug: Frame 0 - 3 clusters found


Processing frames:   1%|          | 2/341 [00:00<02:21,  2.39it/s]

Debug: Frame 1 - 38 persons detected
Debug: Frame 1 - 3 clusters found


...

Debug: Frame 338 - 31 persons detected
Debug: Frame 338 - 4 clusters found
Debug: Frame 338 - Crowd detected (size: 5)
Debug: Frame 338 - Crowd detected (size: 3)


Processing frames: 100%|█████████▉| 340/341 [01:51<00:00,  3.69it/s]

Debug: Frame 339 - 31 persons detected
Debug: Frame 339 - 3 clusters found
Debug: Frame 339 - Crowd detected (size: 6)
Debug: Frame 339 - Crowd detected (size: 3)


Processing frames: 100%|██████████| 341/341 [01:52<00:00,  3.04it/s]

Debug: Frame 340 - 31 persons detected
Debug: Frame 340 - 3 clusters found
Debug: Frame 340 - Crowd detected (size: 5)
Debug: Frame 340 - Crowd detected (size: 3)
Debug: Video capture ended at frame 341 (ret=False)
Debug: CSV file 'crowd_detection.csv' generated with crowd detections
Debug: Script completed processing
